## DeepWalk Methodology

DeepWalk is an approach used to represent the nodes of a graph in an n-dimensional vector space, capturing the social relations within the graph. It leverages local information in the network by employing Random Walks to traverse the graph.

In a Random Walk, you start from a specific node and then randomly choose one of its neighbors to add to the path. This process continues for a desired number of steps. Let's illustrate this with an example.

Consider the directed and unweighted graph shown above. Suppose we start a Random Walk at Node A, and the length of the walk is set to 4. From Node A, we can visit Node B, resulting in the path [A, B]. From Node B, we can choose to move to Node E or even go back to Node A. If we choose to go to Node E, the path becomes [A, B, E]. From Node E, we can further choose to go to Node F or Node C. Let's say we opt for Node F, creating a Random Walk of length 4, like this: **[A, B, E, F]**.

Each Random Walk is essentially a sequence of nodes, akin to a sentence in text, where each node serves as a word in the sentence.

In DeepWalk, you generate Random Walks of a fixed length "l" a specific number of times "m" from each node. This process results in a total of "number_of_nodes * m" sequences, each of length "l". These sequences can be used in conjunction with the skip-gram approach to train a Word2Vec model.

For more insights on Word2Vec, you can refer to JayAlammar's blog post on [Word2Vec](https://jalammar.github.io/illustrated-word2vec/).

---

In [1]:
# Import necessary libraries and modules

import pandas as pd
import numpy as np
from pecanpy import node2vec
from gensim.models import Word2Vec
from Constants import Constants

# Import libraries for progress tracking, joblib, etc.
from tqdm import tqdm
from joblib import load, dump

# Initialize constants and variables

# The name of the graph file
GRAPH_FILE_NAME = Constants.GRAPH_FILE_NAME.value

# Constants for configuring the graph construction
WEIGHTED = Constants.WEIGHTED.value  # Weighted or unweighted graph
DIRECTED = Constants.DIRECTED.value  # Directed or undirected graph

# Number of walks to generate
NUM_WALK = Constants.NUM_WALK.value

# Length of each walk
WALK_LEN = Constants.WALK_LEN.value

# Print the initialized constants and variables for reference
print(GRAPH_FILE_NAME, NUM_WALK, WALK_LEN)


undirected_weighted_product_views_graph.parquet 10 50


---

### Let us consider undirected_weighted_product_views_graph to Generate RandomWalk Sequences

In [2]:
# Specify the path to the graph file using the GRAPH_FILE_NAME
graph_path = '../Data/ConstructedGraph/{}'.format(GRAPH_FILE_NAME)

# Read the graph data from the specified Parquet file.
graph = pd.read_parquet(graph_path)

print("Graph Read from {}".format(graph_path))


Graph Read from ../Data/ConstructedGraph/undirected_weighted_product_views_graph.parquet


In [3]:
graph

,pid_1,pid_2,occurence_ct
0,3601512,3601269,37
1,100059274,3601306,12
2,5100855,4804056,880
3,100041977,100041934,2
4,100057579,100057560,11
...,...,...,...
6846587,12300752,4803895,1
6846588,100017063,6501098,1
6846589,8800741,1005195,1
6846590,25200369,23900098,1


In [4]:
# Calculate the number of unique nodes in the graph
Node_Count = len(set(graph['pid_1'].unique()).union(graph['pid_2'].unique()))

# Calculate the total number of edges in the graph
Edge_Count = len(graph)

print("Graph {} has Nodes: {} | Edges: {}".format(GRAPH_FILE_NAME, Node_Count, Edge_Count))


Graph undirected_weighted_product_views_graph.parquet has Nodes: 211861 | Edges: 6846592


#### Save Graph in .edg format

In [5]:
# Specify the path for the '.edg' graph file, based on the GRAPH_FILE_NAME
edg_graph_path = '../Data/Edg_Graphs_DataFile/' + GRAPH_FILE_NAME.split('.')[0] + '.edg'

# Save the graph data to the '.edg' file, using a tab ('\t') as the separator.
# The 'index' and 'header' are set to False to exclude the index and column names.
graph.to_csv(edg_graph_path, sep='\t', index=False, header=False)

print("Edg Graph Saved")


Edg Graph Saved


### Graph Random Walk Generation

In [6]:
# Initialize a Node2Vec model for deep walk

# 'p' and 'q' are parameters controlling the breadth-first search (BFS) and depth-first search (DFS) behavior.
# In this configuration, 'p' is set to 1, indicating that BFS and DFS are equally important.

g = node2vec.SparseOTF(p=1, q=1, workers=-1, verbose=True, extend=False)


In [23]:
# Measure the execution time of the following code using %%time
%time 

# Read the edge graph from the specified '.edg' file using the initialized Node2Vec model.
# The 'weighted' and 'directed' parameters are configured based on the constants WEIGHTED and DIRECTED.
g.read_edg(edg_graph_path, weighted=WEIGHTED, directed=DIRECTED)

# After reading the edge graph, simulate random walks using the Node2Vec model.
# 'NUM_WALK' specifies the number of walks, and 'WALK_LEN' specifies the length of each walk.
walks = g.simulate_walks(num_walks=NUM_WALK, walk_length=WALK_LEN)


Thread #  6 progress: |###                      | 9.99 %
Thread #  3 progress: |###                      | 9.99 %
Thread #  7 progress: |###                      | 9.99 %
Thread #  5 progress: |###                      | 9.99 %
Thread #  8 progress: |###                      | 9.99 %
Thread #  0 progress: |###                      | 9.99 %
Thread #  1 progress: |###                      | 9.99 %
Thread #  9 progress: |###                      | 9.99 %
Thread #  4 progress: |###                      | 9.99 %
Thread # 11 progress: |###                      | 9.99 %
Thread #  2 progress: |###                      | 9.99 %
Thread # 10 progress: |###                      | 9.99 %
Thread #  6 progress: |#####                    | 19.99 %
Thread #  3 progress: |#####                    | 19.99 %
Thread #  5 progress: |#####                    | 19.99 %
Thread #  7 progress: |#####                    | 19.99 %
Thread #  9 progress: |#####                    | 19.99 %
Thread #  1 progress: |###

In [24]:
# Assert that the number of generated random walks matches the expected value
# If the assertion is true, it means that the correct number of random walks has been generated.

assert len(walks) == Node_Count * NUM_WALK


In [18]:
print(walks[0])

['21405226', '21402696', '21403660', '21401562', '21401454', '21401223', '21400508', '1005105', '1004249', '1002544', '1002524', '5100816', '100042795', '5100476', '5100463', '5100577', '100084968', '10100767', '10100052', '10100054', '10100767', '10101219', '10100825', '100040048', '10400618', '7203417', '8000320', '8000040', '8000303', '8000003', '8000351', '100012123', '7900702', '7101466', '7101666', '7101898', '7101926', '7102070', '7101666', '7101819', '7101816', '7101976', '100064427', '7002254', '7001735', '7002066', '7002254', '7005191', '7003686', '7003804', '6902600']


In [26]:
%%time

# model = Word2Vec(sentences=walks, vector_size=128, window=7, workers=-1, negative=10)

# Initialize a Word2Vec model with the given parameters.
model = Word2Vec(walks,          # Input: Previously generated word walks
                 hs=1,           # Hierarchical Softmax is used for training
                 sg=1,           # Skip-gram model is employed
                 vector_size=128,  # Size of the word embeddings
                 window=5,       # Maximum distance between the current and predicted word within a sentence
                 min_count=1,    # Minimum count of a word in the dataset to be considered for training
                 workers=-1,     # Number of CPU cores to use for training (-1 means use all available cores)
                 seed=42)        # Seed for reproducible results


CPU times: user 9h 11min 54s, sys: 3min 21s, total: 9h 15min 15s
Wall time: 2h 26min 56s


In [27]:
# Save the Word2Vec model to a file with a specific naming convention.
model.save('../Model/deepwalk_graph_embedding_' + GRAPH_FILE_NAME.split('.')[0] + '.model')

In [28]:
# Create a set of unique 'pid' values by combining the 'pid_1' and 'pid_2' columns from the 'graph' DataFrame.
pid_set = set(graph['pid_1'].unique()).union(graph['pid_2'].unique())

# Initialize an empty list to store 'pid' and corresponding 'embedding_vector'.
payload = []

# Iterate through the 'pid_set' and retrieve embedding vectors from the Word2Vec model.
# Use tqdm to display a progress bar.
for pid in tqdm(pid_set):
    try:
        # Append a dictionary containing 'pid' and its corresponding 'embedding_vector' to the 'payload' list.
        payload.append({'pid': pid, 'embedding_vector': model.wv[str(pid]})
    except:
        # If the 'pid' is not in the model's vocabulary, print a message indicating it doesn't exist.
        print(pid, "Not Exist")
        pass


100%|██████████| 211861/211861 [00:01<00:00, 201291.01it/s]


In [29]:
# Create a DataFrame 'embedding_df' from the 'payload' list, containing 'pid' and 'embedding_vector' information.
embedding_df = pd.DataFrame(payload)

# Save the 'embedding_df' DataFrame to a Parquet file with a specific naming convention.
# The 'index' is not included in the saved file.
embedding_df.to_parquet('../Data/Embedding_Data/deepwalk_embedding_df_' + GRAPH_FILE_NAME.split('.')[0] + '.parquet', index=False)


---

## Node2vec

Setting q > 1 will favor a BFS like exploration, while setting q < 1 will favor a DFS like exploration. Hope that helps.

In [7]:
# Initialize a node2vec graph using the SparseOTF (Sparse Online Training Framework) algorithm.
g = node2vec.SparseOTF(p=1,        # Return parameter 'p' for the node2vec random walk
                      q=0.5,      # In-out parameter 'q' for the node2vec random walk
                      workers=-1,  
                      verbose=True,  
                      extend=True)  # Extend the graph data structure for online training


In [8]:
%%time

# Read the edge data from the file specified by 'edg_graph_path' into the node2vec graph 'g'.
# The 'weighted' and 'directed' flags control how the graph is constructed.
g.read_edg(edg_graph_path, weighted=WEIGHTED, directed=DIRECTED)

# Simulate random walks on the graph to generate 'walks'.
# 'num_walks' controls the number of walks to perform, and 'walk_length' is the length of each walk.
walks = g.simulate_walks(num_walks=NUM_WALK, walk_length=WALK_LEN)


Thread #  2 progress: |###                      | 9.99 %
Thread #  4 progress: |###                      | 9.99 %
Thread #  8 progress: |###                      | 9.99 %
Thread #  7 progress: |###                      | 9.99 %
Thread #  3 progress: |###                      | 9.99 %
Thread #  9Thread #  progress: |###                      | 9.99 %
 5 progress: |###                      | 9.99 %
Thread #  6 progress: |###                      | 9.99 %
Thread # 10 progress: |###                      | 9.99 %
Thread #  1 progress: |###                      | 9.99 %
Thread # 11 progress: |###                      | 9.99 %
Thread #  0 progress: |###                      | 9.99 %
Thread #  4 progress: |#####                    | 19.99 %
Thread #  2 progress: |#####                    | 19.99 %
Thread #  3 progress: |#####                    | 19.99 %
Thread #  7 progress: |#####                    | 19.99 %
Thread #  8 progress: |#####                    | 19.99 %
Thread #  5 progress: |###

In [9]:
# Assert that the total number of generated walks ('walks') is equal to the expected value,
# which is calculated as the product of 'Node_Count' and 'NUM_WALK'.
assert len(walks) == Node_Count * NUM_WALK

In [10]:
print(walks[0])

['11500381', '11500310', '11500720', '11500159', '4804295', '5100816', '5100817', '100041309', '5100536', '5100597', '100041309', '5100818', '5100816', '5100875', '5100719', '5100564', '5100577', '100015591', '5100895', '100015591', '5100876', '5100888', '5100860', '5100852', '3601283', '3601605', '3601603', '3600661', '3601495', '3601632', '3601638', '3601015', '3600980', '3600661', '3600937', '3600666', '3600937', '3600661', '3600666', '3601347', '3600182', '3601489', '3601537', '3600937', '3601437', '3601421', '3601286', '3601454', '3600999', '3601514', '2702331']


In [11]:
%%time

# Create a Word2Vec model using the generated 'walks' dataset.
model = Word2Vec(walks,             # Input: Previously generated walks
                 hs=1,              # Use hierarchical softmax for training
                 sg=1,              # Use skip-gram model for training
                 vector_size=128,  # Set the size of the word embeddings
                 window=5,          # Maximum distance between the current and predicted word within a sentence
                 min_count=1,       # Minimum count of a word in the dataset to be considered for training
                 workers=4,         # Number of CPU cores to use for training (adjust as needed)
                 seed=42)           # Set a seed for reproducible results


CPU times: user 4h 27min 53s, sys: 1min 1s, total: 4h 28min 55s
Wall time: 1h 9min 22s


In [12]:
# Save the trained Word2Vec model to a file
model.save('../Model/node2vec_graph_embedding_' + GRAPH_FILE_NAME.split('.')[0] + '.model')


In [13]:
# Create a set 'pid_set' by combining the unique 'pid' values from 'pid_1' and 'pid_2' columns in the 'graph' DataFrame.
pid_set = set(graph['pid_1'].unique()).union(graph['pid_2'].unique())

# Initialize an empty list 'payload' to store 'pid' and corresponding 'embedding_vector'.
payload = []

# Iterate through the 'pid_set' and attempt to retrieve the embedding vectors from the Word2Vec model.
# The 'tqdm' is used to display a progress bar.
for pid in tqdm(pid_set):
    try:
        # Append a dictionary containing 'pid' and its corresponding 'embedding_vector' to the 'payload' list.
        payload.append({'pid': pid, 'embedding_vector': model.wv[str(pid]})
    except:
        # If the 'pid' is not found in the model's vocabulary, print a message indicating it doesn't exist.
        print(pid, "Not Exist")
        pass


100%|██████████| 211861/211861 [00:01<00:00, 182278.49it/s]


In [14]:
# Create a DataFrame 'embedding_df' from the 'payload' list, which contains 'pid' and 'embedding_vector' information.
embedding_df = pd.DataFrame(payload)

# Save the 'embedding_df' DataFrame to a Parquet file with a specific naming convention.
# The 'index' is not included in the saved file.
embedding_df.to_parquet('../Data/Embedding_Data/node2vec_embedding_df_' + GRAPH_FILE_NAME.split('.')[0] + '.parquet', index=False)


In [15]:
embedding_df.head()

,pid,embedding_vector
0,17301504,"[0.44851154, 0.40735474, -0.172185, 0.07374085..."
1,17301505,"[0.47892618, 0.20487863, 0.021507299, -0.62510..."
2,17301506,"[0.25423482, 0.47845665, -0.001093431, 0.03616..."
3,17301507,"[-0.10483544, 0.22859468, -0.16854355, 0.21888..."
4,17301508,"[0.83427334, 0.5198858, 0.011705197, 0.4507888..."


---